# initial Function

In [1]:
! pip install -qq datasets transformers[sentencepiece] simpletransformers
! pip install -qq simpletransformers
#Data management
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import cv2
import re
import string
import os
from tqdm import tqdm
import glob

#Model management
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import torch
import joblib

#data visualize
import matplotlib.pyplot as plt
import seaborn as sns

#image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

#torch
import torch as T
import torch.nn as N
import torch.optim as O

     |████████████████████████████████| 311 kB 5.4 MB/s 
     |████████████████████████████████| 3.5 MB 41.3 MB/s 
     |████████████████████████████████| 248 kB 50.4 MB/s 
     |████████████████████████████████| 243 kB 35.9 MB/s 
     |████████████████████████████████| 1.1 MB 49.2 MB/s 
     |████████████████████████████████| 133 kB 38.1 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
     |████████████████████████████████| 6.8 MB 5.0 MB/s 
     |████████████████████████████████| 1.2 MB 23.5 MB/s 
     |████████████████████████████████| 1.7 MB 32.9 MB/s 
     |████████████████████████████████| 9.7 MB 36.8 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 
     |████████████████████████████████| 596 kB 36.1 MB/s 
     |████████████████████████████████| 180 kB 38.6 MB/s 
     |████████████████████████████████| 144 kB 37.1 MB/s 
     |████████████████████████████████| 63 kB 823 kB/s 
     |████████████████

In [2]:
!nvidia-smi

Wed Feb 16 14:15:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# get Data

In [3]:
! wget https://github.com/kobkrit/datasets/raw/main/AIFORTHAI-LST20Corpus.tar.gz
! tar -xvzf /content/AIFORTHAI-LST20Corpus.tar.gz

Streaming output truncated to the last 5000 lines.
LST20_Corpus/train/._T12371.txt
LST20_Corpus/train/T12371.txt
LST20_Corpus/train/._T11678.txt
LST20_Corpus/train/T11678.txt
LST20_Corpus/train/._T10566.txt
LST20_Corpus/train/T10566.txt
LST20_Corpus/train/._T00775.txt
LST20_Corpus/train/T00775.txt
LST20_Corpus/train/._T12417.txt
LST20_Corpus/train/T12417.txt
LST20_Corpus/train/._T12403.txt
LST20_Corpus/train/T12403.txt
LST20_Corpus/train/._T10214.txt
LST20_Corpus/train/T10214.txt
LST20_Corpus/train/._T01319.txt
LST20_Corpus/train/T01319.txt
LST20_Corpus/train/._T00761.txt
LST20_Corpus/train/T00761.txt
LST20_Corpus/train/._T01457.txt
LST20_Corpus/train/T01457.txt
LST20_Corpus/train/._T00991.txt
LST20_Corpus/train/T00991.txt
LST20_Corpus/train/._T11644.txt
LST20_Corpus/train/T11644.txt
LST20_Corpus/train/._T00749.txt
LST20_Corpus/train/T00749.txt
LST20_Corpus/train/._T13053.txt
LST20_Corpus/train/T13053.txt
LST20_Corpus/train/._T11122.txt
LST20_Corpus/train/T11122.txt
LST20_Corpus/train/

In [4]:
from datasets import load_dataset
lst20 = load_dataset("lst20", data_dir="/content/LST20_Corpus")

Downloading:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Using custom data configuration default-97621787f5b60e1b


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset lst20 downloaded and prepared to /root/.cache/huggingface/datasets/lst20/default-97621787f5b60e1b/0.0.0/e1b2a921fb011578ab43ddbbf789f3c500d62cb2df8ae4ed4b60bae8e4c0d3ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
lst20

DatasetDict({
    train: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 63310
    })
    validation: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 5620
    })
    test: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 5250
    })
})

In [6]:
df_train = pd.DataFrame(lst20["train"])
df_val = pd.DataFrame(lst20["validation"])
df_test = pd.DataFrame(lst20["test"])

# get Embedding layer From HuggingFace


In [7]:
from transformers import AutoTokenizer,AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased')
model_hug = AutoModelForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased',
                                                            revision='finetuned@lst20-pos')


Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/883k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399M [00:00<?, ?B/s]

In [8]:
word_em_initial = model_hug.roberta.embeddings.word_embeddings.weight.detach().numpy()

In [9]:
dict_word2int = tokenizer.vocab # ไว้ใช่้สำหรับแปลง คำเป็ลเลข
vocal_all = list(dict_word2int.keys()) #ใช้สำหรับตรวจคำ
vocab_len = word_em_initial.shape[0] # 25005
emb_dim =  word_em_initial.shape[1] # 768
tokenizer #token

PreTrainedTokenizerFast(name_or_path='airesearch/wangchanberta-base-att-spm-uncased', vocab_size=25005, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': ['<s>NOTUSED', '</s>NOTUSED', '<_>']})

In [10]:
np_emb_weight = model_hug.roberta.embeddings.word_embeddings.weight.detach().numpy()

# encode labels

In [11]:
class Getdata():
  def __init__(self,df,y_col):
    self.df = df[["id","fname","tokens",y_col]]
  
  def turn2int(self,dict_word2int,all_volcab):
    group_paragrape = self.df[self.x_col]
    num_para = group_paragrape.shape[0]
    tokens_list = []
    # loop map ค่าใน dict
    for paragrape in tqdm(group_paragrape):
      list_int = []
      for word in paragrape:
        if word==" ": #clear ช่องว่างออกก่อน
          list_int.append(10)

        elif word in dict_word: #ถ้าเจอใน ditionnary
          list_int.append(word2int[word])
          
        else: # ถ้าไม่เจอใน dictionary ก็ค่อยตัดคำ
          for i in tokenizer(word)["input_ids"]:
            if i not in [5,6,10]:
              list_int.append(i)
              break
      
      tokens_list.append(list_int)
    self.df_int = pd.DataFrame(tokens_list,colums=[self.x_col])

In [12]:
get_train = Getdata(df_train,"pos_tags")

In [13]:


def turn2int(self,dict_word2int,all_volcab,tokenizer):
  group_paragrape = self.df[self.x_col]
  num_para = group_paragrape.shape[0]
  tokens_list = []
  # loop map ค่าใน dict
  for paragrape in tqdm(group_paragrape):
    list_int = []
    for word in paragrape:
      if word==" ": #clear ช่องว่างออกก่อน
        list_int.append(10)
      elif word in all_volcab: #ถ้าเจอใน ditionnary
        list_int.append(dict_word2int[word])
        
      else: # ถ้าไม่เจอใน dictionary ก็ค่อยตัดคำ
        for i in tokenizer(word)["input_ids"]:
          if i not in [5,6,10]:
            list_int.append(i)
            break
    
    tokens_list.append(list_int)

# Preprocessing Batch 32

นี่คือการทดลองทำ Batch 32

In [67]:
BATCH_SIZE = 128

## func

In [68]:
def encode_label(text) :
  payload = {'AJ' : 0, 'AV' : 1, 'AX' :2, 'CC' : 3, 'CL' : 4, 'FX' : 5, 'IJ' : 6, 'NG' : 7, 'NN' : 8, 'NU' : 9, 'O' : 10, 'PA' : 11, 'PR' : 12, 'PS' :13, 'PU' :14, 'VV' :15, 'XX' : 16}
  return payload[text]
def decode_label(text) :
  payload = { 0 : 'AJ', 1 : 'AV',  2:'AX' ,  3 : 'CC',  4 : 'CL', 5 : 'FX', 6 : 'IJ', 7 : 'NG', 8 : 'NN', 9 : 'NU', 10 : 'O', 11 : 'PA',  12 : 'PR', 13 :'PS',  14:'PU',  15:'VV', 16 : 'XX'}
  return payload[text]

In [69]:
dict_word2int = tokenizer.vocab # ไว้ใช่้สำหรับแปลง คำเป็ลเลข
vocab_len = word_em_initial.shape[0] # 25005
emb_dim =  word_em_initial.shape[1] # 768
tokenizer #token

# loop map ค่าใน dict
def endcode_word(word):
  if word==" ": #clear ช่องว่างออกก่อน
    return 10

  elif word in dict_word2int: #ถ้าเจอใน ditionnary
    return dict_word2int[word]

  # else:
  #   return 3

  else: # ถ้าไม่เจอใน dictionary ก็ค่อยตัดคำ
    for i in tokenizer(word)["input_ids"]:
      if i not in [5,6,10]:
        return i

## train Data

In [70]:
words_payload = []
labels_payload = []

for i in glob.glob('/content/LST20_Corpus/train/*') :
  fp = open(i,"r")
  for line in fp.readlines() :
    if line == '\n' :
      continue
    else :
      data_ = line[:-2].split("\t")
      words_payload.append(data_[0])
      labels_payload.append(data_[1])

In [71]:
%%time
words_payload_encoded = [endcode_word(r) for r in tqdm(words_payload)]
labels_payload_encoded = [encode_label(r) for r in tqdm(labels_payload)]

100%|██████████| 2714726/2714726 [00:02<00:00, 986582.39it/s] 

CPU times: user 27.9 s, sys: 293 ms, total: 28.2 s
Wall time: 28.8 s


In [72]:
np_word_encoded = np.array(words_payload_encoded)
np_label_encoded = np.array(labels_payload_encoded)

In [73]:
batched_word = np_word_encoded[:-1*(np_word_encoded.shape[0]%BATCH_SIZE)].reshape(-1,BATCH_SIZE)
batched_label = np_label_encoded[:-1*(np_label_encoded.shape[0]%BATCH_SIZE)].reshape(-1,BATCH_SIZE)

In [74]:
print(batched_word)
print(batched_label)
batched_word = batched_word.astype('float32')
batched_label = batched_label.astype('float32')

[[5693 566 15 ... 265 4705 265]
 [10331 265 851 ... 265 18 16395]
 [18 265 3827 ... 1262 58 592]
 ...
 [79 75 144 ... 265 18 3742]
 [265 16721 18 ... 13271 20 8373]
 [22 265 414 ... 670 126 21]]
[[ 8 15  2 ... 14 13 14]
 [ 8 14 15 ... 14 14  8]
 [14 14  8 ...  8  5 15]
 ...
 [ 8  8  8 ... 14 14  8]
 [14  8 14 ...  8 13  8]
 [ 2 14  0 ...  8 15  3]]


## validate data

In [75]:
val_words_payload = []
val_labels_payload = []

for i in glob.glob('/content/LST20_Corpus/eval/*') :
  fp = open(i,"r")
  for line in fp.readlines() :
    if line == '\n' :
      continue
    else :
      data_ = line[:-2].split("\t")
      val_words_payload.append(data_[0])
      val_labels_payload.append(data_[1])

In [76]:
%%time
val_words_payload_encoded = [endcode_word(r) for r in tqdm(val_words_payload)]
val_labels_payload_encoded = [encode_label(r) for r in tqdm(val_labels_payload)]

100%|██████████| 240860/240860 [00:00<00:00, 905327.94it/s]

CPU times: user 2.66 s, sys: 37.9 ms, total: 2.7 s
Wall time: 2.71 s


In [77]:
np_val_word_encoded = np.array(val_words_payload_encoded)
np_val_label_encoded = np.array(val_labels_payload_encoded)

In [78]:
val_batched_word = np_val_word_encoded[:-1*(np_val_word_encoded.shape[0]%BATCH_SIZE)].reshape(-1,BATCH_SIZE)
val_batched_label = np_val_label_encoded[:-1*(np_val_label_encoded.shape[0]%BATCH_SIZE)].reshape(-1,BATCH_SIZE)

In [79]:
print(val_batched_word.shape)
print(val_batched_label.shape)
val_batched_word = batched_word.astype('float32')
val_batched_label = batched_label.astype('float32')

(1881, 128)
(1881, 128)


## test data

In [80]:
test_words_payload = []
test_labels_payload = []

for i in glob.glob('/content/LST20_Corpus/test/*') :
  fp = open(i,"r")
  for line in fp.readlines() :
    if line == '\n' :
      continue
    else :
      data_ = line[:-2].split("\t")
      test_words_payload.append(data_[0])
      test_labels_payload.append(data_[1])

In [81]:
%%time
test_words_payload_encoded = [endcode_word(r) for r in tqdm(test_words_payload)]
test_labels_payload_encoded = [encode_label(r) for r in tqdm(test_labels_payload)]

100%|██████████| 207278/207278 [00:00<00:00, 966027.36it/s]

CPU times: user 2 s, sys: 36.1 ms, total: 2.04 s
Wall time: 2.05 s


In [82]:
np_test_word_encoded = np.array(test_words_payload_encoded)
np_test_label_encoded = np.array(test_labels_payload_encoded)

In [83]:
test_batched_word = np_test_word_encoded[:-1*(np_test_word_encoded.shape[0]%BATCH_SIZE)].reshape(-1,BATCH_SIZE)
test_batched_label = np_test_label_encoded[:-1*(np_test_label_encoded.shape[0]%BATCH_SIZE)].reshape(-1,BATCH_SIZE)

In [84]:
print(test_batched_word.shape)
print(test_batched_label.shape)
test_batched_word = batched_word.astype('float32')
test_batched_label = batched_label.astype('float32')

(1619, 128)
(1619, 128)


# Create Model

In [85]:
vocab_len = 25005
emb_dim = 768
unique_y = 20

In [86]:
def LSTM(vocab_len,emb_dim):
    inputs = layers.Input(shape=(BATCH_SIZE))

    Emb = layers.Embedding(trainable=False,input_dim = vocab_len, output_dim = emb_dim ,name="emb")(inputs)

    blstm = layers.Bidirectional(layers.LSTM(256, return_sequences=True, dropout = 0.2))(Emb)
    blstm = layers.Bidirectional(layers.LSTM(256, return_sequences=True, dropout = 0.2))(blstm)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(blstm)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(x)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(x)
    x = layers.TimeDistributed(layers.Dense(17, activation = 'softmax'))(x)

    act_model = Model(inputs, x)
    act_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    print(act_model.summary())
    return act_model

In [87]:
model = LSTM(vocab_len,emb_dim)
np_emb_weight = np_emb_weight.reshape(1,25005,768)
model.get_layer("emb").set_weights(np_emb_weight) # use pretain weight

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 128)]             0         
                                                                 
 emb (Embedding)             (None, 128, 768)          19203840  
                                                                 
 bidirectional_10 (Bidirecti  (None, 128, 512)         2099200   
 onal)                                                           
                                                                 
 bidirectional_11 (Bidirecti  (None, 128, 512)         1574912   
 onal)                                                           
                                                                 
 time_distributed_13 (TimeDi  (None, 128, 128)         65664     
 stributed)                                                      
                                                           

In [ ]:
ES = EarlyStopping(monitor='val_loss', patience=3, verbose=1,min_delta=0,restore_best_weights=True)
model.fit(batched_word , 
          batched_label , 
          epochs=10,
          validation_data=(val_batched_word,val_batched_label),
          callbacks = [ES],
          batch_size = 128)

Epoch 1/10
166/166 [==============================] - 75s 455ms/step - loss: 0.3892 - accuracy: 0.8824 - val_loss: 0.1558 - val_accuracy: 0.9514
Epoch 2/10
166/166 [==============================] - 73s 443ms/step - loss: 0.1532 - accuracy: 0.9513 - val_loss: 0.1280 - val_accuracy: 0.9585
Epoch 3/10
166/166 [==============================] - 73s 443ms/step - loss: 0.1352 - accuracy: 0.9561 - val_loss: 0.1178 - val_accuracy: 0.9610
Epoch 4/10
 68/166 [===========>..................] - ETA: 32s - loss: 0.1262 - accuracy: 0.9586

In [ ]:
model.evaluate(test_batched_word,test_batched_label)

# Dowload True Test

In [ ]:
!gdown --id 1cwlaGd-l1yFqT7LctDqxT5ikpJj0vx_g

In [ ]:
!unzip /content/pos_test.txt.zip -d /content/submission/

In [ ]:
fp = open("/content/submission/pos_test.txt","r")

In [ ]:
word_list_submission = []
for line in fp.readlines() :
  if line != '\n' :
    word_list_submission.append(line[:-1])
  else :
    word_list_submission.append('_')

In [ ]:
print(len(word_list_submission))
word_list_submission = word_list_submission[:-1]
len(word_list_submission)

In [ ]:
word_list_submission += ['_']*23

In [ ]:
word_list_submission_encoded = [endcode_word(r) for r in tqdm(word_list_submission)]

In [ ]:
np_word_list_submission_encoded_batched = np.array(word_list_submission_encoded).reshape(-1,BATCH_SIZE)

In [ ]:
np_word_list_submission_encoded_batched.shape

In [ ]:
np_word_list_submission_encoded_batched

# Predict True test

In [ ]:
z = model.predict(np_word_list_submission_encoded_batched).argmax(axis=2)

In [ ]:
func2_ = np.vectorize(lambda x : decode_label(x))
z_decode = func2_(z)
z_decode = z_decode.flatten()
z_decode = z_decode.tolist()[:-23]
print(len(z_decode))
print(z_decode[:50])

# Submission

In [ ]:
submission_df = pd.read_csv("/content/pos_sample_submission.csv")

In [ ]:
submission_df.Predicted = z_decode

In [ ]:
submission_df.to_csv("submission.csv",index=False)

In [ ]:
model.save('LSTM_emb_bert.h5')